In [1]:
import pandas as pd
from sqlalchemy import create_engine, text

# ⚠️ adapte le driver si besoin: "ODBC Driver 17" ou "ODBC Driver 18"
ENGINE = create_engine(
    "mssql+pyodbc://@localhost/DWH?driver=ODBC+Driver+17+for+SQL+Server;Trusted_Connection=yes"
)
engine = ENGINE  # alias pratique


In [2]:
# Vérif rapide des colonnes disponibles
df = pd.read_sql("SELECT * FROM ml.vw_features;", engine)
print(df.shape)
df.head(3)
list(df.columns)


DBAPIError: (pyodbc.Error) ('IM012', '[IM012] [Microsoft][ODBC Driver Manager] DRIVER keyword syntax error (0) (SQLDriverConnect)')
(Background on this error at: https://sqlalche.me/e/20/dbapi)

In [3]:
import pyodbc
pyodbc.drivers()


['SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 17 for SQL Server',
 'ODBC Driver 18 for SQL Server']

In [4]:
import urllib.parse
from sqlalchemy import create_engine

driver = "ODBC Driver 18 for SQL Server"   # <- remplace si ton print(pyodbc.drivers()) montre "ODBC Driver 17 for SQL Server"
params = urllib.parse.quote_plus(
    f"DRIVER={driver};"
    "SERVER=localhost;"
    "DATABASE=DWH;"
    "Trusted_Connection=yes;"
    "Encrypt=yes;"
    "TrustServerCertificate=yes"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}", fast_executemany=True)


In [5]:
with engine.connect() as cn:
    print(cn.exec_driver_sql("SELECT @@VERSION").scalar())


Microsoft SQL Server 2022 (RTM) - 16.0.1000.6 (X64) 
	Oct  8 2022 05:58:25 
	Copyright (C) 2022 Microsoft Corporation
	Developer Edition (64-bit) on Windows 10 Pro 10.0 <X64> (Build 19045: )



In [6]:
import pandas as pd
df = pd.read_sql("SELECT TOP 5 * FROM ml.vw_features;", engine)
df.head()


,opportunite_key,opportunite_id_source,label_won,is_train,montant_offre_tnd,nb_reunions,satisfaction_score,nb_reclamations,cycle_vente_jours,secteur,taille_entreprise,produit_cat,region,equipe
0,1,OPP-2000,0,1,350116.0,6,3,2,95,Santé,Moyenne,Autre,Nabeul,Équipe Santé
1,2,OPP-2001,1,1,174061.0,7,1,4,126,Microfinance,Moyenne,Sécurité,Sfax,Équipe Finance
2,3,OPP-2002,0,1,182944.0,4,4,4,37,Microfinance,Moyenne,Cloud,Sfax,Équipe Santé
3,4,OPP-2003,0,1,98202.0,6,2,1,44,Microfinance,Grande,Datacom,Nabeul,Équipe Santé
4,5,OPP-2004,0,0,438865.0,1,1,3,74,Banque,Grande,Datacom,Tunis,Équipe Finance


In [7]:
import pandas as pd

# 1. Charger toutes les features
df = pd.read_sql("SELECT * FROM ml.vw_features;", engine)
print(df.shape)
df.head(3)

# 2. Colonnes
target_col = "label_won"            # la cible (0/1)
id_cols    = ["opportunite_key", "opportunite_id_source", "is_train"]  # colonnes à garder mais pas pour l'entraînement

# numériques (prises dans ta vue) :
num_cols = [
    "montant_offre_tnd","nb_reunions","satisfaction_score",
    "nb_reclamations","cycle_vente_jours"
]

# catégorielles (one-hot) :
cat_cols = ["secteur", "taille_entreprise", "produit_cat", "region", "equipe"]

# Vérif de présence
missing = [c for c in num_cols+cat_cols+[target_col]+id_cols if c not in df.columns]
print("Colonnes manquantes :", missing)


(500, 14)
Colonnes manquantes : []


In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Split train / score à partir de is_train
df_train = df[df["is_train"]==1].copy()
df_score = df[df["is_train"]==0].copy()

X_train = df_train[num_cols + cat_cols]
y_train = df_train[target_col].astype(int)

# Prépro : scaler sur numériques, one-hot sur catégorielles
prepro = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols),
    ],
    remainder="drop",
)

# Modèle simple et robuste
clf = LogisticRegression(max_iter=200, n_jobs=None)

pipe = Pipeline([
    ("prep", prepro),
    ("clf", clf),
])


In [9]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, classification_report

# (optionnel) petite CV rapide AUC
auc = cross_val_score(pipe, X_train, y_train, cv=5, scoring="roc_auc").mean()
print(f"AUC CV (5-fold) ≈ {auc:.3f}")

# Fit sur tout le train
pipe.fit(X_train, y_train)

# Perf sur train (diagnostic rapide)
proba_train = pipe.predict_proba(X_train)[:,1]
print("AUC train :", roc_auc_score(y_train, proba_train))
print(classification_report(y_train, (proba_train>=0.5).astype(int)))


AUC CV (5-fold) ≈ 0.537
AUC train : 0.6225361068216912
              precision    recall  f1-score   support

           0       0.59      0.49      0.53       202
           1       0.59      0.68      0.63       218

    accuracy                           0.59       420
   macro avg       0.59      0.58      0.58       420
weighted avg       0.59      0.59      0.58       420



In [10]:
import numpy as np
from datetime import date

# Scoring
X_score = df_score[num_cols + cat_cols]
proba = pipe.predict_proba(X_score)[:,1]

# Seuil binaire (0.5 par défaut)
seuil = 0.5
pred_bin = (proba >= seuil).astype(int)

# date_key au format AAAAMMJJ (à mapper ensuite sur ta dim_temps si nécessaire)
date_key = int(date.today().strftime("%Y%m%d"))

# Version du modèle (texte libre)
model_version = "logreg_v1"

# DataFrame conforme à ml.fact_opportunite_prediction
df_pred = pd.DataFrame({
    "opportunite_key": df_score["opportunite_key"].values.astype(int),
    "date_prediction_key": date_key,
    "proba_predite": proba.astype(float),
    "model_version": model_version,
    "seuil_utilise": seuil,
    "prediction_binaire": pred_bin.astype(int),
})

df_pred.head()


,opportunite_key,date_prediction_key,proba_predite,model_version,seuil_utilise,prediction_binaire
0,5,20251015,0.558354,logreg_v1,0.5,1
1,8,20251015,0.630917,logreg_v1,0.5,1
2,13,20251015,0.432896,logreg_v1,0.5,0
3,41,20251015,0.569440,logreg_v1,0.5,1
4,58,20251015,0.508795,logreg_v1,0.5,1


In [11]:
with engine.begin() as cn:
    # vide (sécurisé si table existe)
    cn.exec_driver_sql("TRUNCATE TABLE ml.fact_opportunite_prediction;")

# insert (append)
df_pred.to_sql(
    name="fact_opportunite_prediction",
    schema="ml",
    con=engine,
    if_exists="append",
    index=False
)

# Vérif
check = pd.read_sql("SELECT COUNT(*) AS n FROM ml.fact_opportunite_prediction;", engine)
check


IntegrityError: (pyodbc.IntegrityError) ('23000', '[23000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]The INSERT statement conflicted with the FOREIGN KEY constraint "FK_pred_date". The conflict occurred in database "DWH", table "dwh.dim_temps", column \'date_key\'. (547) (SQLExecute); [23000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]The statement has been terminated. (3621); [23000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]The INSERT statement conflicted with the FOREIGN KEY constraint "FK_pred_date". The conflict occurred in database "DWH", table "dwh.dim_temps", column \'date_key\'. (547); [23000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]The statement has been terminated. (3621); [23000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]The INSERT statement conflicted with the FOREIGN KEY constraint "FK_pred_date". The conflict occurred in database "DWH", table "dwh.dim_temps", column \'date_key\'. (547); [23000] [Microsoft][ODBC

In [12]:
# 1) essayer la date du jour si elle existe dans la dimension
dk_today = pd.read_sql("""
SELECT TOP 1 date_key
FROM dwh.dim_temps
WHERE [date] = CAST(GETDATE() AS date)
""", engine)

if dk_today.empty:
    # 2) sinon, prendre la plus récente dispo
    dk_today = pd.read_sql("SELECT MAX(date_key) AS date_key FROM dwh.dim_temps", engine)

dk = int(dk_today.iloc[0,0])

# Remplacer la colonne par cette clé valide
df_pred["date_prediction_key"] = dk

# Écrire après avoir vidé la table (si tu veux écraser)
with engine.begin() as cn:
    cn.exec_driver_sql("TRUNCATE TABLE ml.fact_opportunite_prediction;")

df_pred.to_sql(
    name="fact_opportunite_prediction",
    schema="ml",
    con=engine,
    if_exists="append",
    index=False
)


-1

In [13]:
check = pd.read_sql("SELECT COUNT(*) AS n FROM ml.fact_opportunite_prediction;", engine)
check


,n
0,80


In [14]:
df_pred.to_csv("predictions_opportunites.csv", index=False, encoding="utf-8")


In [15]:
# Récupérer la date_key du jour (ou d’une date choisie)
today_key = pd.read_sql("""
SELECT TOP 1 date_key 
FROM dwh.dim_temps 
WHERE full_date = CAST(GETDATE() AS date)
""", engine).iloc[0,0]

df_pred["date_prediction_key"] = today_key


ProgrammingError: (pyodbc.ProgrammingError) ('42S22', "[42S22] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Invalid column name 'full_date'. (207) (SQLExecDirectW)")
[SQL: 
SELECT TOP 1 date_key 
FROM dwh.dim_temps 
WHERE full_date = CAST(GETDATE() AS date)
]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [16]:
# 1) Construire la date_key du jour au format AAAAMMJJ côté SQL Server
today_key = pd.read_sql(
    "SELECT CAST(CONVERT(char(8), GETDATE(), 112) AS int) AS date_key;",
    engine
).iloc[0, 0]

# (Option: pour une date choisie)
# target = '2025-09-22'
# today_key = pd.read_sql(
#     f"SELECT CAST(CONVERT(char(8), CAST('{target}' AS date), 112) AS int) AS date_key;",
#     engine
# ).iloc[0, 0]

# 2) Affecter cette clé à toutes les lignes à insérer
df_pred["date_prediction_key"] = int(today_key)


In [17]:
with engine.begin() as cn:
    # si tu veux refaire les prédictions du jour, nettoie d'abord ce jour-là
    cn.exec_driver_sql(f"DELETE FROM ml.fact_opportunite_prediction WHERE date_prediction_key = {int(today_key)};")

df_pred.to_sql(
    name="fact_opportunite_prediction",
    schema="ml",
    con=engine,
    if_exists="append",
    index=False
)


IntegrityError: (pyodbc.IntegrityError) ('23000', '[23000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]The INSERT statement conflicted with the FOREIGN KEY constraint "FK_pred_date". The conflict occurred in database "DWH", table "dwh.dim_temps", column \'date_key\'. (547) (SQLExecute); [23000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]The statement has been terminated. (3621); [23000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]The INSERT statement conflicted with the FOREIGN KEY constraint "FK_pred_date". The conflict occurred in database "DWH", table "dwh.dim_temps", column \'date_key\'. (547); [23000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]The statement has been terminated. (3621); [23000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]The INSERT statement conflicted with the FOREIGN KEY constraint "FK_pred_date". The conflict occurred in database "DWH", table "dwh.dim_temps", column \'date_key\'. (547); [23000] [Microsoft][ODBC

In [18]:
pd.read_sql("""
SELECT TOP 10 date_key
FROM dwh.dim_temps
ORDER BY date_key DESC
""", engine)


,date_key
0,20250922
1,20250921
2,20250920
3,20250919
4,20250918
5,20250917
6,20250916
7,20250915
8,20250914
9,20250913


In [19]:
# clé "au format int yyyymmdd"
want_key = pd.read_sql("SELECT CONVERT(int, FORMAT(GETDATE(),'yyyyMMdd')) AS k", engine).iloc[0,0]

# si want_key n’existe pas dans dwh.dim_temps, on prend la plus récente existante
today_key = pd.read_sql(f"""
SELECT COALESCE(
  (SELECT date_key FROM dwh.dim_temps WHERE date_key = {int(want_key)}),
  (SELECT MAX(date_key) FROM dwh.dim_temps)
) AS date_key
""", engine).iloc[0,0]

print("today_key utilisé :", today_key)


today_key utilisé : 20250922


In [20]:
df_pred["date_prediction_key"] = int(today_key)

with engine.begin() as cn:
    cn.exec_driver_sql(f"""
        DELETE FROM ml.fact_opportunite_prediction
        WHERE date_prediction_key = {int(today_key)};
    """)

df_pred.to_sql(
    name="fact_opportunite_prediction",
    schema="ml",
    con=engine,
    if_exists="append",
    index=False
)


-1

In [21]:
pd.read_sql("""
SELECT COUNT(*) AS n FROM ml.fact_opportunite_prediction;
SELECT COUNT(*) AS n FROM ml.fact_opportunite_prediction
WHERE date_prediction_key = (SELECT MAX(date_prediction_key) FROM ml.fact_opportunite_prediction);
""", engine)


,n
0,80


In [23]:
# --- Connexion SQLAlchemy déjà créée en 'engine' ---

import pandas as pd

# a) Charger les features
df = pd.read_sql("SELECT * FROM ml.vw_features;", engine)

# b) Split train/score
num_cols = ["montant_offre_tnd","nb_reunions","satisfaction_score","nb_reclamations","cycle_vente_jours"]
cat_cols = ["secteur","taille_entreprise","produit_cat","region","equipe"]

X_train = df[df["is_train"]==1][num_cols+cat_cols]
y_train = df[df["is_train"]==1]["label_won"].astype(int)
X_score = df[df["is_train"]==0][num_cols+cat_cols]
keys    = df[df["is_train"]==0][["opportunite_key"]]

# c) Pipeline simple (ex.: logreg)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pre = ColumnTransformer([
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])

pipe = Pipeline([
    ("pre", pre),
    ("clf", LogisticRegression(max_iter=1000))
])

pipe.fit(X_train, y_train)

# d) Scoring
proba = pipe.predict_proba(X_score)[:,1]
seuil = 0.5
pred_bin = (proba >= seuil).astype(int)

# e) Récupérer une date_key valide (la plus récente <= aujourd'hui)
today_key = pd.read_sql("""
    SELECT MAX(date_key) AS date_key
    FROM dwh.dim_temps
    WHERE date_key <= CONVERT(int, FORMAT(GETDATE(),'yyyyMMdd'))
""", engine).iloc[0,0]


# f) Préparer DF d’insertion
df_pred = pd.DataFrame({
    "opportunite_key": keys["opportunite_key"].values,
    "date_prediction_key": int(today_key),
    "proba_predite": proba.astype(float),
    "model_version": "logreg_v1",
    "seuil_utilise": seuil,
    "prediction_binaire": pred_bin.astype(int)
})

# g) Nettoyer puis insérer (idempotent pour ce jour)
with engine.begin() as cn:
    cn.exec_driver_sql(f"""
        DELETE FROM ml.fact_opportunite_prediction
        WHERE date_prediction_key = {int(today_key)};
    """)

df_pred.to_sql(
    name="fact_opportunite_prediction",
    schema="ml",
    con=engine,
    if_exists="append",
    index=False
)


-1

In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
from sqlalchemy.engine import Engine

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report


In [6]:
# Créer l'engine seulement si non défini
try:
    engine  # vérifie si existe
except NameError:
    DRIVER   = "ODBC Driver 18 for SQL Server"
    SERVER   = "localhost\\SQLEXPRESS"  # adapte si besoin
    DATABASE = "DWH"
    TRUST    = "yes"  # TrustServerCertificate
    conn_str = (
        f"mssql+pyodbc:///?Driver={DRIVER};"
        f"Server={SERVER};Database={DATABASE};"
        f"Trusted_Connection=Yes;TrustServerCertificate={TRUST}"
    )
    engine: Engine = create_engine(conn_str, fast_executemany=True)


In [5]:
# Créer l'engine seulement si non défini
try:
    engine  # vérifie si existe
except NameError:
    DRIVER   = "ODBC Driver 18 for SQL Server"
    SERVER   = "localhost\\SQLEXPRESS"  # adapte si besoin
    DATABASE = "DWH"
    TRUST    = "yes"  # TrustServerCertificate
    conn_str = (
        f"mssql+pyodbc:///?Driver={DRIVER};"
        f"Server={SERVER};Database={DATABASE};"
        f"Trusted_Connection=Yes;TrustServerCertificate={TRUST}"
    )
    engine: Engine = create_engine(conn_str, fast_executemany=True)


In [7]:
df = pd.read_sql("SELECT * FROM ml.vw_features;", engine)
print("Shape:", df.shape)
df.head(3)


InterfaceError: (pyodbc.InterfaceError) ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')
(Background on this error at: https://sqlalche.me/e/20/rvf5)

In [8]:
try:
    with engine.connect() as cn:
        print("OK ->", cn.exec_driver_sql("SELECT @@VERSION").scalar()[:60], "…")
except Exception as e:
    print("ENGINE KO ->", e)


ENGINE KO -> (pyodbc.InterfaceError) ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')
(Background on this error at: https://sqlalche.me/e/20/rvf5)


In [9]:
from sqlalchemy import create_engine

DRIVER   = "ODBC Driver 18 for SQL Server"   # adapte si tu as le 17
SERVER   = r"localhost"                      # ou NOM_MACHINE\INSTANCE
DATABASE = "DWH"

conn_str = (
    f"mssql+pyodbc:///?Driver={DRIVER};"
    f"Server={SERVER};Database={DATABASE};"
    "Trusted_Connection=Yes;TrustServerCertificate=Yes"
)

engine = create_engine(conn_str, fast_executemany=True, pool_pre_ping=True)


C:\Users\g50048083\AppData\Local\Temp\ipykernel_8420\3113563792.py:13: SAWarning: No driver name specified; this is expected by PyODBC when using DSN-less connections
  engine = create_engine(conn_str, fast_executemany=True, pool_pre_ping=True)


In [10]:
with engine.connect() as cn:
    print(cn.exec_driver_sql("SELECT DB_NAME()").scalar())


InterfaceError: (pyodbc.InterfaceError) ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')
(Background on this error at: https://sqlalche.me/e/20/rvf5)

In [11]:
from sqlalchemy import create_engine
import urllib.parse

params = {
    "DRIVER": "{ODBC Driver 18 for SQL Server}",  # or "{ODBC Driver 17 for SQL Server}" if 18 isn't installed
    "SERVER": r"localhost",                       # or "localhost\SQLEXPRESS" etc.
    "DATABASE": "DWH",
    "Trusted_Connection": "Yes",
    "TrustServerCertificate": "Yes",
}
odbc_str = ";".join(f"{k}={v}" for k, v in params.items())
engine = create_engine(
    f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(odbc_str)}",
    fast_executemany=True,
    pool_pre_ping=True,
)


In [12]:
with engine.connect() as cn:
    print(cn.exec_driver_sql("SELECT DB_NAME()").scalar())


DWH


In [13]:
import pandas as pd

df = pd.read_sql("SELECT * FROM ml.vw_features;", engine)
print("Shape:", df.shape)
df.head(3)


Shape: (500, 14)


,opportunite_key,opportunite_id_source,label_won,is_train,montant_offre_tnd,nb_reunions,satisfaction_score,nb_reclamations,cycle_vente_jours,secteur,taille_entreprise,produit_cat,region,equipe
0,1,OPP-2000,0,1,350116.0,6,3,2,95,Santé,Moyenne,Autre,Nabeul,Équipe Santé
1,2,OPP-2001,1,1,174061.0,7,1,4,126,Microfinance,Moyenne,Sécurité,Sfax,Équipe Finance
2,3,OPP-2002,0,1,182944.0,4,4,4,37,Microfinance,Moyenne,Cloud,Sfax,Équipe Santé


In [14]:
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import cross_val_score

# --- colonnes ---
num_cols = [
    "montant_offre_tnd","nb_reunions","satisfaction_score",
    "nb_reclamations","cycle_vente_jours"
]
# certaines colonnes catégorielles peuvent ne pas exister selon ta vue -> on filtre
possible_cat = ["secteur","taille","produit_cat","region","equipe"]
cat_cols = [c for c in possible_cat if c in df.columns]

feat_cols = num_cols + cat_cols

# --- split ---
X_train = df.loc[df["is_train"]==1, feat_cols]
y_train = df.loc[df["is_train"]==1, "label_won"].astype(int)

X_score = df.loc[df["is_train"]==0, feat_cols]          # à « scorer » (pas d’étiquettes)

# --- préprocesseur + modèle ---
preproc = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols),
    ],
    remainder="drop",
)

model = LogisticRegression(max_iter=200, n_jobs=None)   # simple & robuste

pipe = Pipeline(steps=[("prep", preproc), ("clf", model)])

# --- entraînement ---
pipe.fit(X_train, y_train)

# --- diag rapide ---
proba_train = pipe.predict_proba(X_train)[:, 1]
print("AUC train :", roc_auc_score(y_train, proba_train))
print(classification_report(y_train, (proba_train >= 0.5).astype(int)))


AUC train : 0.6159278771913888
              precision    recall  f1-score   support

           0       0.58      0.50      0.54       202
           1       0.59      0.66      0.62       218

    accuracy                           0.58       420
   macro avg       0.58      0.58      0.58       420
weighted avg       0.58      0.58      0.58       420



In [15]:
seuil = 0.5

# Probas sur train
pred_train = (proba_train >= seuil).astype(int)

# Probas sur le « score set »
proba_score = pipe.predict_proba(X_score)[:, 1]
pred_score  = (proba_score >= seuil).astype(int)

# Petit aperçu : on garde la clé pour joindre plus tard
apercu_train = df.loc[df["is_train"]==1, ["opportunite_key"]].copy()
apercu_train["proba_predite"] = proba_train
apercu_train["prediction_binaire"] = pred_train
print("Aperçu TRAIN :")
display(apercu_train.head())

apercu_score = df.loc[df["is_train"]==0, ["opportunite_key"]].copy()
apercu_score["proba_predite"] = proba_score
apercu_score["prediction_binaire"] = pred_score
print("Aperçu SCORE (à insérer un autre jour) :")
display(apercu_score.head())


Aperçu TRAIN :


,opportunite_key,proba_predite,prediction_binaire
0,1,0.513075,1
1,2,0.615670,1
2,3,0.486795,0
3,4,0.569427,1
5,6,0.640953,1


Aperçu SCORE (à insérer un autre jour) :


,opportunite_key,proba_predite,prediction_binaire
4,5,0.593862,1
7,8,0.622520,1
12,13,0.400076,0
34,41,0.610909,1
47,58,0.455916,0


In [16]:
import pandas as pd
df = pd.read_sql("SELECT * FROM ml.vw_features;", engine)
print("Shape:", df.shape)
df.head(3)


Shape: (500, 14)


,opportunite_key,opportunite_id_source,label_won,is_train,montant_offre_tnd,nb_reunions,satisfaction_score,nb_reclamations,cycle_vente_jours,secteur,taille_entreprise,produit_cat,region,equipe
0,1,OPP-2000,0,1,350116.0,6,3,2,95,Santé,Moyenne,Autre,Nabeul,Équipe Santé
1,2,OPP-2001,1,1,174061.0,7,1,4,126,Microfinance,Moyenne,Sécurité,Sfax,Équipe Finance
2,3,OPP-2002,0,1,182944.0,4,4,4,37,Microfinance,Moyenne,Cloud,Sfax,Équipe Santé


In [17]:
train = df[df["is_train"] == 1].copy()
score = df[df["is_train"] == 0].copy()

y_train = train["label_won"].astype(int)
X_train = train.drop(columns=["label_won", "opportunite_id_source", "is_train"])
X_score = score.drop(columns=["label_won", "opportunite_id_source", "is_train"])
keys = score[["opportunite_key"]].copy()


In [18]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

num_cols = X_train.select_dtypes(include="number").columns.tolist()
cat_cols = X_train.select_dtypes(exclude="number").columns.tolist()

preproc = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols)
    ],
    remainder="drop"
)

model = LogisticRegression(max_iter=200, n_jobs=None)  # simple et rapide
pipe = Pipeline([("prep", preproc), ("clf", model)])
pipe.fit(X_train, y_train)


Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['opportunite_key',
                                                   'montant_offre_tnd',
                                                   'nb_reunions',
                                                   'satisfaction_score',
                                                   'nb_reclamations',
                                                   'cycle_vente_jours']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['secteur',
                                                   'taille_entreprise',
                                                   'produit_cat', 'region',
                                                   'equipe'])])),
                ('clf', LogisticRegression(max_iter=200))])

In [19]:
from sklearn.metrics import roc_auc_score, classification_report

proba_train = pipe.predict_proba(X_train)[:, 1]
seuil = 0.5
pred_train = (proba_train >= seuil).astype(int)

print("AUC train :", roc_auc_score(y_train, proba_train))
print(classification_report(y_train, pred_train))


AUC train : 0.6225815242074666
              precision    recall  f1-score   support

           0       0.59      0.50      0.54       202
           1       0.59      0.68      0.63       218

    accuracy                           0.59       420
   macro avg       0.59      0.59      0.59       420
weighted avg       0.59      0.59      0.59       420



In [20]:
proba = pipe.predict_proba(X_score)[:, 1]
pred_bin = (proba >= seuil).astype(int)

apercu_train = pd.DataFrame({
    "opportunite_key": train["opportunite_key"].values,
    "proba_predite": proba_train,
    "prediction_binaire": pred_train
})
apercu_score = pd.DataFrame({
    "opportunite_key": keys["opportunite_key"].values,
    "proba_predite": proba,
    "prediction_binaire": pred_bin
})

display(apercu_train.head())
display(apercu_score.head())


,opportunite_key,proba_predite,prediction_binaire
0,1,0.549451,1
1,2,0.654533,1
2,3,0.523805,1
3,4,0.526223,1
4,6,0.682029,1


,opportunite_key,proba_predite,prediction_binaire
0,5,0.552041,1
1,8,0.626706,1
2,13,0.427297,0
3,41,0.564760,1
4,58,0.503779,1


In [21]:
import joblib
joblib.dump(pipe, "modele_logreg_v1.joblib")


['modele_logreg_v1.joblib']

In [22]:
# paramètres "version"
model_version = "logreg_v1"
seuil = 0.5

# a) récupérer une date_key valide depuis dwh.dim_temps
today_key = pd.read_sql("""
    SELECT MAX(date_key) AS date_key
    FROM dwh.dim_temps
    WHERE [date] <= CAST(GETDATE() AS date)
""", engine).iloc[0,0]
print("date_prediction_key utilisé :", today_key)

# b) préparer le DataFrame pour l’insert
df_pred = pd.DataFrame({
    "opportunite_key": keys["opportunite_key"].values,
    "date_prediction_key": int(today_key),
    "proba_predite": proba.astype(float),
    "model_version": model_version,
    "seuil_utilise": seuil,
    "prediction_binaire": pred_bin.astype(int),
})
df_pred.head()


date_prediction_key utilisé : 20250922


,opportunite_key,date_prediction_key,proba_predite,model_version,seuil_utilise,prediction_binaire
0,5,20250922,0.552041,logreg_v1,0.5,1
1,8,20250922,0.626706,logreg_v1,0.5,1
2,13,20250922,0.427297,logreg_v1,0.5,0
3,41,20250922,0.564760,logreg_v1,0.5,1
4,58,20250922,0.503779,logreg_v1,0.5,1


In [23]:
with engine.begin() as cn:
    cn.exec_driver_sql(f"""
        DELETE FROM ml.fact_opportunite_prediction
        WHERE date_prediction_key = {int(today_key)};
    """)

df_pred.to_sql(
    name="fact_opportunite_prediction",
    schema="ml",
    con=engine,
    if_exists="append",
    index=False
)


-1

In [26]:
# nombre total de lignes
pd.read_sql("SELECT COUNT(*) AS n FROM ml.fact_opportunite_prediction;", engine)

# nombre de lignes insérées pour le jour courant
pd.read_sql(f"""
SELECT COUNT(*) AS n
FROM ml.fact_opportunite_prediction
WHERE date_prediction_key = {int(today_key)};
""", engine)

# répartition 0/1 pour le jour courant
pd.read_sql(f"""
SELECT prediction_binaire, COUNT(*) AS n
FROM ml.fact_opportunite_prediction
WHERE date_prediction_key = {int(today_key)}
GROUP BY prediction_binaire
ORDER BY prediction_binaire;
""", engine).style.hide_index()


AttributeError: 'Styler' object has no attribute 'hide_index'

In [25]:
pd.read_sql(f"""
SELECT prediction_binaire, COUNT(*) AS n
FROM ml.fact_opportunite_prediction
WHERE date_prediction_key = {int(today_key)}
GROUP BY prediction_binaire
ORDER BY prediction_binaire;
""", engine)


,prediction_binaire,n
0,False,44
1,True,36


In [29]:
# nombre total de lignes
pd.read_sql("SELECT COUNT(*) AS n FROM ml.fact_opportunite_prediction;", engine)

# nombre de lignes insérées pour le jour courant
pd.read_sql(f"""
SELECT COUNT(*) AS n
FROM ml.fact_opportunite_prediction
WHERE date_prediction_key = {int(today_key)};
""", engine)

# répartition 0/1 pour le jour courant
pd.read_sql(f"""
SELECT prediction_binaire, COUNT(*) AS n
FROM ml.fact_opportunite_prediction
WHERE date_prediction_key = {int(today_key)}
GROUP BY prediction_binaire
ORDER BY prediction_binaire;
""", engine).style.hide_index()


AttributeError: 'Styler' object has no attribute 'hide_index'

In [28]:
pd.read_sql(f"""
SELECT prediction_binaire, COUNT(*) AS n
FROM ml.fact_opportunite_prediction
WHERE date_prediction_key = {int(today_key)}
GROUP BY prediction_binaire
ORDER BY prediction_binaire;
""", engine).style.hide(axis="index")


prediction_binaire,n
False,44
True,36


In [30]:
# nombre total de lignes
pd.read_sql("SELECT COUNT(*) AS n FROM ml.fact_opportunite_prediction;", engine)

# nombre de lignes insérées pour le jour courant
pd.read_sql(f"""
SELECT COUNT(*) AS n
FROM ml.fact_opportunite_prediction
WHERE date_prediction_key = {int(today_key)};
""", engine)

# répartition 0/1 pour le jour courant
pd.read_sql(f"""
SELECT prediction_binaire, COUNT(*) AS n
FROM ml.fact_opportunite_prediction
WHERE date_prediction_key = {int(today_key)}
GROUP BY prediction_binaire
ORDER BY prediction_binaire;
""", engine).style.hide_index()


AttributeError: 'Styler' object has no attribute 'hide_index'

In [31]:
# 3a) Nombre de prédictions du jour
nb_total = pd.read_sql(f"""
SELECT COUNT(*) AS n
FROM ml.fact_opportunite_prediction
WHERE date_prediction_key = {int(today_key)}
""", engine)
print(nb_total.to_string(index=False))  # affichage sans index

# 3b) Répartition 0/1 pour le jour courant
rep = pd.read_sql(f"""
SELECT prediction_binaire, COUNT(*) AS n
FROM ml.fact_opportunite_prediction
WHERE date_prediction_key = {int(today_key)}
GROUP BY prediction_binaire
ORDER BY prediction_binaire;
""", engine)

# option: ajouter le pourcentage
rep["pct"] = rep["n"] / rep["n"].sum()

print("\nRépartition :")
print(rep.to_string(index=False))  # pas de .style du tout


 n
80

Répartition :
 prediction_binaire  n  pct
              False 44 0.55
               True 36 0.45


In [32]:
apercu = pd.read_sql(f"""
SELECT TOP 10 *
FROM ml.fact_opportunite_prediction
WHERE date_prediction_key = {int(today_key)}
ORDER BY opportunite_key
""", engine)
display(apercu)


,predict_id,opportunite_key,date_prediction_key,proba_predite,model_version,seuil_utilise,prediction_binaire
0,1002,5,20250922,0.552041,logreg_v1,0.5,True
1,1003,8,20250922,0.626706,logreg_v1,0.5,True
2,1004,13,20250922,0.427297,logreg_v1,0.5,False
3,1005,41,20250922,0.564760,logreg_v1,0.5,True
4,1006,58,20250922,0.503779,logreg_v1,0.5,True
5,1007,89,20250922,0.672025,logreg_v1,0.5,True
6,1008,94,20250922,0.418859,logreg_v1,0.5,False
7,1009,111,20250922,0.581233,logreg_v1,0.5,True
8,1010,113,20250922,0.530666,logreg_v1,0.5,True
9,1011,120,20250922,0.645513,logreg_v1,0.5,True


In [33]:
dups = pd.read_sql(f"""
SELECT opportunite_key, date_prediction_key, COUNT(*) AS n
FROM ml.fact_opportunite_prediction
WHERE date_prediction_key = {int(today_key)}
GROUP BY opportunite_key, date_prediction_key
HAVING COUNT(*) > 1
""", engine)

print("Doublons trouvés :", len(dups))
display(dups.head())


Doublons trouvés : 0


,opportunite_key,date_prediction_key,n


In [34]:
stats = pd.read_sql(f"""
SELECT 
  MIN(proba_predite) AS min_proba,
  MAX(proba_predite) AS max_proba,
  AVG(proba_predite) AS avg_proba,
  SUM(CASE WHEN prediction_binaire=1 THEN 1 ELSE 0 END) AS nb_pred_1,
  SUM(CASE WHEN prediction_binaire=0 THEN 1 ELSE 0 END) AS nb_pred_0
FROM ml.fact_opportunite_prediction
WHERE date_prediction_key = {int(today_key)}
""", engine)
display(stats)


,min_proba,max_proba,avg_proba,nb_pred_1,nb_pred_0
0,0.266056,0.757243,0.496264,36,44


In [36]:
rep_secteur = pd.read_sql(f"""
SELECT f.secteur, COUNT(*) AS n,
       AVG(CASE WHEN p.prediction_binaire=1 THEN 1.0 ELSE 0 END) AS part_1
FROM ml.vw_features f
JOIN ml.fact_opportunite_prediction p
  ON p.opportunite_key = f.opportunite_key
WHERE p.date_prediction_key = {int(today_key)}
GROUP BY f.secteur
ORDER BY n DESC
""", engine)
display(rep_secteur)


,secteur,n,part_1
0,Banque,24,0.250000
1,Microfinance,20,0.450000
2,Assurance,19,0.894736
3,Santé,17,0.235294
